## Analise da base de dados `Beta` utilizando algoritmos de ML

Neste notebook será analisado o `Beta dataset` utilizando algoritmos de ML para realizar a: 

1. Extração de características;
2. Seleção de características;
3. Classificação dos dados.

### Pontos importantes do dataset

- Frequências estimuladas (total de 40, com diferença de 0.2Hz uma da outra): 8.0; 8.2;...; 15.6; 15.8;
- Taxa de amostragem: 250Hz

### Analisar os "momentos" em que ocorrem evocação do sinal SSVEP
1. Criar o objeto `MNE` a partir dos dados do participante;
2. Aplicar no objeto `MNE` o filtro passa-faixa nos valores de 6 a 18 Hz;
3. Criar cópias do objeto `MNE` com fatias de tempo menores para analisar momentos que ocorrem estimulos ou não (verificar artigo);

> 0.0 - 0.5 segundos e 2.5 - 3.0 segundos, ocorre apenas ruído; <br/>
 0.5 - 2.5 segundos, ocorre sinal SSVEP (com ruídos)
 
4. Com os sinais separados em objetos `MNE`, aplicar a `FFT`, para que seja possível plotar gráficos que contenham (ou não) as informações:
- Os dados devem ser plotados do domínio da frequência (após a transformada de Fourier). O `FFT` pode ser realizado pela biblioteca `scipy.fft`.
- Deve ser observado que as janelas (a) com ruído não aparecerão de fato o sinal SSVEP.

In [165]:
# Imports

import pandas as pd
import sklearn as skl
import matplotlib as plt
import mne
import numpy as np
from scipy.io import loadmat
from sklearn.feature_selection import RFE, RFECV
from sklearn.svm import SVC, SVR
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, classification_report
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

# beta = loadmat(f"../../datasets/beta/S13.mat")
beta = np.load(f"../../datasets/beta/data_beta.npy")

### Extração de características

Uma característica importante de acordo com o artigo base do dataset `Beta` é o *signal-to-noise ratio* (SNR).

> Incluir equação SNR

Ao finalizar essa etapa, será obtido um vetor de características. Essas podem ser:
- `SNR` (obrigatória);
- Maior valor espectral (FFT em vez do compute_psd);
- Média dos valores espectrais (FFT).

Dimensionalidade dos dados será explicada da seguinte forma:
- `40, 4, 64, 750` -> 40 targets, 4 trials, 64 canais, 750 valores 
- `160, 64 (SNR) + 64 (media) + 64 (maior) ...` Resultando em `160, 192` 

In [166]:
mne_data = mne.read_epochs("mne_data_beta.fif")
beta = mne_data.get_data()
# mne_data

Reading /home/vinicius/Documents/RP_SSVEP/ssvep/src/beta/mne_data_beta.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    2996.00 ms
        0 CTF compensation matrices available


Not setting metadata
160 matching events found
No baseline correction applied
0 projection items activated


> Agora iremos estimar o ruído de fundo, para calcular posteriormente o `narrow SNR` e o `wide-band SNR`.

In [167]:
# Estimando o ruído de fundo

# intervalos de tempo sem estímulo (0 a 0,5 segundos e 2,5 a 3 segundos)
base_start = 0
base_end = 125
rest_start = 625
rest_end = 750

# armazena uma lista com as médias de potência para cada canal
noise_power = []

# consideramos a primeira amostra (1º target, 1º trial)
for channel_data in beta[0, :, :]:
    fft_result = np.fft.fft(channel_data)
    
    # densidade espectral de potência (PSD)
    psd = np.abs(fft_result) ** 2
    
    # média da potência nos intervalos de tempo sem estímulo
    base_power = np.mean(psd[base_start:base_end])
    rest_power = np.mean(psd[rest_start:rest_end])
    
    # média das duas médias de potência obtidas anteriormente
    mean_noise_power = (base_power + rest_power) / 2
    noise_power.append(mean_noise_power)
    
#média das médias de potência de todos os canais para estimar o ruído de fundo
estimated_background_noise = np.mean(noise_power)
estimated_background_noise

30465.602510101286

> Antes de calcular os SNRs, precisamos obter as amplitudes alvo por meio dos dados EEG:

In [168]:
from scipy.signal import find_peaks

sr = 250

# frequências alvo
target_frequencies = np.arange(8, 16, 0.2)

# lista para armazenar as amplitudes nas frequências alvo
target_amplitudes = []

# for channel_data in X_beta[0, :, :]:
for channel_data in mne_data.get_data():
    target = []
    for eletrodo in channel_data:
        # fft_result = np.fft.fft(channel_data)
        fft_result = np.fft.fft(eletrodo)
        psd = np.abs(fft_result) ** 2
        frequencies = np.fft.fftfreq(len(fft_result), 1 / sr)
        target_amplitudes_trial = []
        for target_frequency in target_frequencies:
            # encontrando o índice da frequência alvo no espectro de frequência
            index = np.argmin(np.abs(frequencies - target_frequency))
            # amplitude na frequência alvo
            amplitude = np.sqrt(psd[index])
            target_amplitudes_trial.append(amplitude)
        
        target.append(target_amplitudes_trial)

    target_amplitudes.append(target)
    
target_amplitudes = np.array(target_amplitudes)
target_amplitudes.shape

(160, 64, 40)

Vamos calcular o SNR de "banda estreita". Pode ser observado pela seguinte equação:

$SNR_{banda\ estreita} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{média das amplitudes nas frequências vizinhas}}\right)$

Já o SNR de banda larga é definido da seguinte forma:

$SNR_{banda\ larga} = 10 \cdot \log_{10}\left(\frac{\text{energia total do espectro}}{\text{energia total do espectro de amplitude}}\right)$

In [169]:
# forçando (estragando) valor de "estimated_background_noise" para não sobrar valores negativos
# estimated_background_noise = 1.
target_amplitudes_adjusted = target_amplitudes - estimated_background_noise

# subtraindo o ruído de fundo das amplitudes
narrow_band_SNR = 10 * np.log10(target_amplitudes_adjusted / estimated_background_noise)
# print(narrow_band_SNR)
print(narrow_band_SNR.shape)

total_power = np.sum(target_amplitudes_adjusted)
wide_band_SNR = 10 * np.log10(target_amplitudes_adjusted / total_power)
# print(wide_band_SNR)
print(wide_band_SNR.shape)


X = np.array([narrow_band_SNR, wide_band_SNR])
X = X.swapaxes(0, 1)
X = X.swapaxes(1, 2)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2] * X.shape[3])
print()
print(X.shape)

(160, 64, 40)
(160, 64, 40)

(160, 64, 80)


Ao final desta etapa, será obtido um vetor de características. Estas podem ser:
- `narrow SNR` (obrigatória);
- `wide-band SNR` (obrigatória);
- Maior valor espectral (FFT);
- Média dos valores espectrais (FFT).

> Dimensionalidade dos dados será explicada da seguinte forma:

`40, 4, 64, 750` -> 40 targets, 4 trials, 64 canais e 750 valores `160, 64 (SNR) + 64 (média) + 64 (maior) ...`
Resultando em `(160, 192)`.

`(160, 64, 750)` = 2 * `(160, 64, 40)` = `(160, 64, 80)` => (reshape) `(160, 64)`

reshape => `(160, 64, 80)`

> (1) **Seleção de características (MANUAL)**

Remover os eletrodos não listados no artigo => `(160, 9, 80)` => reshape `(160, 720)`

> (2) Vetor de característica para a **seleção de características RFE**

`(160, 64, 80)` => reshape => `(160, 5120)`. Após aplicar o RFE, retorna  `(160, N)`

> (3) Aplicar SVM (kernel='linear'), nas duas seleções

### Seleção de características manualmente

In [170]:
# 1 - Seleção de características (MANUAL)

ch = np.load("../../datasets/beta/channels.npy")

best_ch = ['PZ', 'PO3', 'PO5', 'PO4', 'PO6', 'POZ', 'O1', 'OZ', 'O2']
index_remove = np.where(np.isin(ch, best_ch, invert=True))

X_selected_ch = np.delete(X, index_remove, axis=1)
print(X_selected_ch.shape)

X_selected_ch = X_selected_ch.reshape(X_selected_ch.shape[0], X_selected_ch.shape[1] * X_selected_ch.shape[2])
print(X_selected_ch.shape)


(160, 9, 80)
(160, 720)


### Seleção de todas as características 

In [171]:
# 2 - Vetor de característica para a seleção de características RFE

X_reshape = X.reshape(X.shape[0], X.shape[1] * X.shape[2])
print(X_reshape.shape)

(160, 5120)


### Aplicando SVM para as características selecionadas manualmente

In [172]:
labels = np.load("../../datasets/beta/labels_beta.npy")

X = StandardScaler().fit_transform(X_selected_ch)
y = LabelEncoder().fit_transform(labels)

# Preencher valores NaN com a media da coluna
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imputer.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

svm_model = SVC(kernel='linear').fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

print("Accuracy: %.2f %%" % (accuracy_score(y_test, y_pred) * 100))


Accuracy: 28.12 %


### Aplicando validação cruzada com SVM para as características selecionadas manualmente

In [173]:
labels = np.load("../../datasets/beta/labels_beta.npy")

X = StandardScaler().fit_transform(X_selected_ch)
y = LabelEncoder().fit_transform(labels)


# Preencher valores NaN com a media da coluna
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imputer.fit_transform(X)

skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
acuracias = []

for treino_idx, teste_idx in skf.split(X, y):

    X_treino = X[treino_idx]
    y_treino = y[treino_idx]

    X_teste = X[teste_idx]
    y_teste = y[teste_idx]

    svm_model.fit(X_treino, y_treino)
    y_pred = svm_model.predict(X_teste)

    acuracias.append(accuracy_score(y_teste, y_pred))

print("Accuracy: %.2f %%" % (np.mean(acuracias) * 100))

Accuracy: 16.88 %


### Seleção de características e classificação

Como existem diversos eletrodos (canais) que não obtém sinal SSVEP, podemos extrair as características que não contribuem para a classificação dos dados.

Podemos utilizar o método `RFE` (*Recursive Feature Elimination*) aplicando por meio de `sklearn.feature_selection.RFE`, aprimorando o parâmetro `n_features_to_select` até obter o melhor resultado de classificação.

Para a classificação propriamente dita, é considerado o uso do método `SVM`.

In [174]:
# Preencher valores NaN com a media da coluna
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_reshape = imputer.fit_transform(X_reshape)

# Normalizando o X
X_norm = MinMaxScaler().fit_transform(X_reshape)

# svm_model_full = SVC(kernel='linear', C=1, random_state=0, probability=True)
rfe = RFECV(SVC(kernel="linear"), min_features_to_select=100, step=5, cv=4, scoring='accuracy')
X_full = rfe.fit_transform(X_norm, y)


In [175]:
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X_full, y, test_size=0.2, random_state=0)

svm_full_model = SVC(kernel='linear', C=10, random_state=42, probability=True).fit(X_train_full, y_train_full)
y_full_pred = svm_full_model.predict(X_test_full)

print("Accuracy: %.2f %%" % (accuracy_score(y_test_full, y_full_pred) * 100))

Accuracy: 21.88 %
